# Assignment V2

## Imports

In [ ]:
#checks for environment
import sys
print(sys.executable)

In [ ]:
import pandas as pd
import importlib

In [ ]:
#import all modules including necessary libraries
from lib.openai_client import*
from lib.paths import DATA_DIR,PROJECT_ROOT,CSV_DIR, TXT_DIR, PRODUCTS_JSON_PATH, PRODUCTS_CSV_PATH
from lib.data_loading import load_csv_products, load_txt_products
import lib.constants
importlib.reload(lib.constants)
from lib.text_extraction import extract_products_from_txt_df
from lib.parsing import normalize_csv_products
from lib.validation import add_validation_columns
from lib.catalog import build_catalog_for_llm

import lib.chatbot
import lib.ui_catalog_chat

importlib.reload(lib.chatbot)
importlib.reload(lib.ui_catalog_chat)

from lib.chatbot import create_catalog_system_context
from lib.ui_catalog_chat import build_catalog_chat_dashboard
from lib.export import export_products


## Data loading


Récuperer la donnée des fichiers txt, puis csv et la display

In [ ]:
df_csv_raw = load_csv_products(CSV_DIR)
df_txt_raw = load_txt_products(TXT_DIR)


In [ ]:
df_csv_raw.head()

In [ ]:
df_txt_raw.head()

In [ ]:
print(df_txt_raw.loc[0, "raw_text"])

## CSV File extraction

In [ ]:

df_products_csv = normalize_csv_products(df_csv_raw)

In [ ]:
df_products_csv

## TXT files extraction using api

In [ ]:
df_products_txt = extract_products_from_txt_df(df_txt_raw)

In [ ]:
df_products_txt

## Normalisation and validation

Now that we have the good columns, and that our features are well parsed, let's Join txt and csv data frame

In [ ]:
df_all_products = pd.concat(
    [df_products_csv, df_products_txt],
    ignore_index=True
)

df_all_products


In [ ]:
df_validated = add_validation_columns(df_all_products)

In [ ]:
df_validated.head()

## API use for ChatBot : summarize, translate

In [ ]:

catalog_df = df_validated.copy()  
CATALOG_TEXT = build_catalog_for_llm(catalog_df)

In [ ]:
context = create_catalog_system_context(CATALOG_TEXT)

In [ ]:
dashboard = build_catalog_chat_dashboard(context)
dashboard

## Data export

exporting the data into their file

In [ ]:
json_path, csv_path = export_products(df_validated)